### Gmail OAuth2 authentication 

In [1]:
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlin

ERROR: Could not find a version that satisfies the requirement google-auth-oauthlin (from versions: none)
ERROR: No matching distribution found for google-auth-oauthlin


In [2]:
import os.path

In [3]:
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

In [4]:
import os.path
import base64
import email

In [5]:
SCOPES=['https://www.googleapis.com/auth/gmail.readonly']

In [6]:
import os
from google_auth_oauthlib.flow import InstalledAppFlow
from google.oauth2.credentials import Credentials
from google.auth.transport.requests import Request

SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']

def authenticate_gmail():
    creds = None

    # Checking if token.json (or client.json) already exists
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())  # Refresh the token if expired
        else:
            flow = InstalledAppFlow.from_client_secrets_file('client.json', SCOPES)
            creds = flow.run_local_server(port=0) 

        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    return creds 

creds = authenticate_gmail()


In [7]:
service = build('gmail', 'v1', credentials=creds)

### fetch_emails(query: str) supports flexible search queries

In [8]:
import base64
import pandas as pd
from email import policy
import email

import base64
import pandas as pd
from email import policy
import email

def fetch_emails(service, query='', max_results=100):
    all_emails = []
    next_page_token = None
    fetched = 0

    while True:
        response = service.users().messages().list(
            userId='me',
            q=query, 
            pageToken=next_page_token,
            maxResults=min(100, max_results - fetched)
        ).execute()
        messages = response.get('messages', [])
        if not messages:
            break

        for msg_meta in messages:
            try:
                msg = service.users().messages().get(
                    userId='me',
                    id=msg_meta['id'],
                    format='raw'
                ).execute()
                raw_bytes = base64.urlsafe_b64decode(msg['raw'])
                mime_msg = email.message_from_bytes(raw_bytes, policy=policy.default)
                sender = mime_msg.get('From', '')
                subject = mime_msg.get('Subject', '')
                date = mime_msg.get('Date', '')

                # Extracting body
                body = ""
                if mime_msg.is_multipart():
                    for part in mime_msg.walk():
                        ctype = part.get_content_type()
                        payload = part.get_payload(decode=True)
                        if ctype == 'text/plain' and payload:
                            body = payload.decode(errors='replace')
                            break
                        elif ctype == 'text/html' and payload and not body:
                            body = payload.decode(errors='replace')
                else:
                    payload = mime_msg.get_payload(decode=True)
                    if payload:
                        body = payload.decode(errors='replace')

                all_emails.append([sender, subject, date, body])
                fetched += 1
                if fetched >= max_results:
                    break
            except Exception as e:
                print(f"Error processing message {msg_meta['id']}: {e}")
                continue

        if fetched >= max_results:
            break
        next_page_token = response.get('nextPageToken')
        if not next_page_token:
            break

    df = pd.DataFrame(all_emails, columns=['Sender', 'Subject', 'Date', 'Body'])
    return df

C:\Users\MAITHILI\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\MAITHILI\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


### Trying out with a keyword "interview"

* It will give us a dataframe where interview is in the subject of the mail

In [10]:
emails_df = fetch_emails(service, query='subject:interview', max_results=50)
display(emails_df.head())

,Sender,Subject,Date,Body
0,Citizens <noreply@mail.modernhire.com>,Your Citizens interview link is a click away! ...,"Fri, 25 Apr 2025 20:47:19 +0000","<style type=""text/css"">#EmailBody a {color:#02..."
1,Quora Digest <english-personalized-digest@quor...,How do I get a 3.6 in a Google interview?,"Thu, 24 Apr 2025 17:09:35 +0000",Top stories for Maithili\r\n\r\n-----\r\n\r\nQ...
2,Team Unstop <noreply@unstop.news>,Top Companies are Hiring [2025]! - Ace Your In...,"Thu, 27 Feb 2025 13:33:03 +0530",<!DOCTYPE html>\n\n<html><head><title></title>...
3,"""Atria Convergence Technologies (ACT)"" <udita....",✉️ Walk-in interview | Collection Executive in...,"Mon, 24 Feb 2025 17:49:37 +0530","\r\n<!DOCTYPE html PUBLIC ""-//W3C//DTD XHTML 1..."
4,Katie Warren from TopResume <katie@topresume.com>,Your interview,"Thu, 13 Feb 2025 07:13:06 -0500",A few minutes of this will improve your chance...


### Trying out with a dates

In [11]:
query = 'after:2025/04/01 before:2025/05/01'
emails_df = fetch_emails(service, query=query, max_results=100)
emails_df.head()

,Sender,Subject,Date,Body
0,Caleb Ralphs <caleb.ralphs@valisinsights.com>,Data Science Intern - VALIS Insights,"Wed, 30 Apr 2025 23:24:51 +0000","Hello Maithili,\r\n\r\nThank you for applying ..."
1,Workable <noreply@candidates.workablemail.com>,Thanks for applying to VALIS Insights,"Wed, 30 Apr 2025 22:24:30 +0000",VALIS Insights\r\n\r\n------------------------...
2,"""AncestryRecruiting@ancestry.com"" <AncestryRec...",Thank you for applying to Ancestry!,"Wed, 30 Apr 2025 14:34:03 -0700","<!doctype html><html lang=en xmlns=""http://www..."
3,Stack Sports <do-not-reply@mail.paylocity.com>,Thank you for applying!,"Wed, 30 Apr 2025 21:27:12 +0000","<html style=""background-color: #F4F6F8;""><head..."
4,no-reply@us.greenhouse-mail.io,Thank you for applying to DeepIntent,"Wed, 30 Apr 2025 21:23:04 +0000","Maithili,\r\n\r\nThanks for applying to DeepIn..."


### Cleaning the dataset 

In [12]:
from bs4 import BeautifulSoup
import re

def is_css_or_junk(text):
    t = text.strip()
    if not t:
        return True
    if re.match(r'^([.#]?\w+\s*\{)', t):
        return True
    if len(t) < 40 and all(c in '{};:. \r\n\t' for c in t):
        return True
    lines = t.splitlines()
    if lines:
        css_lines = [l for l in lines if l.strip().endswith('{') or l.strip().endswith('}')]
        if len(css_lines) / len(lines) > 0.7:
            return True
    return False

def clean_html_body(html_body):
    soup = BeautifulSoup(html_body, "html.parser")
    for s in soup(["script", "style"]):
        s.decompose()
    text = soup.get_text(separator="\n", strip=True)
    text = re.sub(r'\n+', '\n', text)
    text = re.sub(r'[ \t]+', ' ', text)
    return text.strip()

def clean_body(text):
    if not isinstance(text, str):
        return ""
    if is_css_or_junk(text):
        return ""
    if '<html' in text.lower() or '<body' in text.lower() or '<div' in text.lower() or '<!doctype' in text.lower():
        return clean_html_body(text)
    return text.strip()

def add_cleaned_body_column(df):
    df['Body_Clean'] = df['Body'].apply(clean_body)
    return df


### Extracting unique email address 

In [13]:
def extract_email_addresses(text):
    if not isinstance(text, str):
        return []
    return list(set(re.findall(r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}', text)))

def add_extracted_emails_column(df):
    def get_emails(row):
        emails = extract_email_addresses(row['Sender']) + extract_email_addresses(row['Body_Clean'])
        return list(set(emails))
    df['Extracted_Emails'] = df.apply(get_emails, axis=1)
    return df

In [14]:
emails_df = add_cleaned_body_column(emails_df)
emails_df.head()

,Sender,Subject,Date,Body,Body_Clean
0,Caleb Ralphs <caleb.ralphs@valisinsights.com>,Data Science Intern - VALIS Insights,"Wed, 30 Apr 2025 23:24:51 +0000","Hello Maithili,\r\n\r\nThank you for applying ...","Hello Maithili,\r\n\r\nThank you for applying ..."
1,Workable <noreply@candidates.workablemail.com>,Thanks for applying to VALIS Insights,"Wed, 30 Apr 2025 22:24:30 +0000",VALIS Insights\r\n\r\n------------------------...,VALIS Insights\r\n\r\n------------------------...
2,"""AncestryRecruiting@ancestry.com"" <AncestryRec...",Thank you for applying to Ancestry!,"Wed, 30 Apr 2025 14:34:03 -0700","<!doctype html><html lang=en xmlns=""http://www...","Hi Maithili,\nThank you for taking the time to..."
3,Stack Sports <do-not-reply@mail.paylocity.com>,Thank you for applying!,"Wed, 30 Apr 2025 21:27:12 +0000","<html style=""background-color: #F4F6F8;""><head...","Dear Maithili,Thank you for your interest in a..."
4,no-reply@us.greenhouse-mail.io,Thank you for applying to DeepIntent,"Wed, 30 Apr 2025 21:23:04 +0000","Maithili,\r\n\r\nThanks for applying to DeepIn...","Maithili,\r\n\r\nThanks for applying to DeepIn..."


In [15]:
emails_df = add_extracted_emails_column(emails_df)
emails_df.head()

,Sender,Subject,Date,Body,Body_Clean,Extracted_Emails
0,Caleb Ralphs <caleb.ralphs@valisinsights.com>,Data Science Intern - VALIS Insights,"Wed, 30 Apr 2025 23:24:51 +0000","Hello Maithili,\r\n\r\nThank you for applying ...","Hello Maithili,\r\n\r\nThank you for applying ...",[caleb.ralphs@valisinsights.com]
1,Workable <noreply@candidates.workablemail.com>,Thanks for applying to VALIS Insights,"Wed, 30 Apr 2025 22:24:30 +0000",VALIS Insights\r\n\r\n------------------------...,VALIS Insights\r\n\r\n------------------------...,"[noreply@candidates.workablemail.com, maithili..."
2,"""AncestryRecruiting@ancestry.com"" <AncestryRec...",Thank you for applying to Ancestry!,"Wed, 30 Apr 2025 14:34:03 -0700","<!doctype html><html lang=en xmlns=""http://www...","Hi Maithili,\nThank you for taking the time to...","[maithili.a7@gmail.com, ancestryrecruiting@anc..."
3,Stack Sports <do-not-reply@mail.paylocity.com>,Thank you for applying!,"Wed, 30 Apr 2025 21:27:12 +0000","<html style=""background-color: #F4F6F8;""><head...","Dear Maithili,Thank you for your interest in a...",[do-not-reply@mail.paylocity.com]
4,no-reply@us.greenhouse-mail.io,Thank you for applying to DeepIntent,"Wed, 30 Apr 2025 21:23:04 +0000","Maithili,\r\n\r\nThanks for applying to DeepIn...","Maithili,\r\n\r\nThanks for applying to DeepIn...",[no-reply@us.greenhouse-mail.io]
